## Stochasticising

This notebook converts each current adjacency matrix into a stochastic transition matrix by converting all rows to probabilities (ie normalising them to 1). We then raise it to the power of 7 – which is equivalent to saying you can move one town per day, and then convert the probabilities back to relative flows and save it as new adjacency matrices (again setting the diagonals as equal to 0), writing them into the folder ‘stochasticised’. It raises a few concerns though; the value of C is super sensitive when trying to run the discrete SEIR model using it, and should we really be setting the diagonal as 0?!

In [1]:
#Import necessary libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Read in the filtered adjacency matrices
wa_297 = os.listdir("/Volumes/HardDrive/kalman_all2all_big/")[:-1]
d = {}
for i in range(len(wa_297)):
    d[str(i)] = pd.read_csv("/Volumes/HardDrive/kalman_all2all_big/"+wa_297[i], header = None, sep ='\t')

In [65]:
def stochasticise(dfs):
    
    df = dfs.copy()
    rowsums = []
    
    # Get the sum of each row, then divide each entry by that sum (normalise)
    for index, row in df.iterrows():
        a = sum(row)
        rowsums.append(a)
        if a > 0:
            for i in df.columns:
                df.iloc[index, i] = df.iloc[index, i]/a
    
    dfc = df.copy()
    # Raise the transition matrix to the power of 7
    for i in range(7):
        df = np.matmul(df,dfc)
    
    # Convert it back to relative flows
    for index, row in df.iterrows():
        for i in df.columns:
            df.iloc[index, i] = df.iloc[index, i]*rowsums[index]
            
    # Set diagonal to 0 (think about whether you want this)        
    for i in range(len(df.columns)):
        df.iloc[i,i]=0
        
    return df

In [168]:
# Run the function over each dataframe and write it to a folder
for i in range(len(wa_297)):
    print(i)
    temp = stochasticise(d[str(i)])
    temp.to_csv("/Volumes/HardDrive/stochasticised/"+wa_297[i],sep = '\t',index = False, header = False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67


## Further Adjustments

In [163]:
def stoch(dfs):
    
    df = dfs.copy()
    rowsums = []
    
    # Get the sum of each row, then divide each entry by that sum (normalise)
    for index, row in df.iterrows():
        a = sum(row)
        rowsums.append(a)
        if a > 0:
            for i in df.columns:
                df.iloc[index, i] = df.iloc[index, i]/a
    
    tot_df = df.copy()/7
    temp_df = df.copy()
    # Raise the transition matrix to the power of 7
    for i in range(6):
        temp_df = np.matmul(temp_df,df)
        tot_df += temp_df/7
    
    selfloop = []
    # Set diagonal to 0  
    for i in range(len(tot_df.columns)):
        selfloop.append(tot_df.iloc[i,i])
        tot_df.iloc[i,i]=0
    
    # Convert it back to relative flows (distribute diagonals evenly throughout the row)
    for index, row in tot_df.iterrows():
        for i in tot_df.columns:
            tot_df.iloc[index, i] = tot_df.iloc[index, i]/(1-selfloop[index])*rowsums[index]
        
    return tot_df

In [164]:
# Run the function over each dataframe and write it to a folder
for i in range(len(wa_297)):
    print(i)
    temp = stoch(d[str(i)])
    temp.to_csv("/Volumes/HardDrive/stochasticised/"+wa_297[i],sep = '\t',index = False, header = False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67


# Checks

In [158]:
rowsums = []
for index, row in po.iterrows():
    a = sum(row)
    rowsums.append(a)

In [159]:
rowsums == ro

False

In [160]:
for i in range(len(rowsums)):
    print(rowsums[i],ro[i],rowsums[i]-ro[i])

0.00012811167991790353 0.00012811167991790353 0.0
2.66345062903406e-06 2.66345062903406e-06 0.0
2.983190018081857e-06 2.9831900180818573e-06 -4.235164736271502e-22
4.7084075283894344e-07 4.708407528389434e-07 5.293955920339377e-23
8.033764508294465e-07 8.033764508294461e-07 3.1763735522036263e-22
2.4096643946915874e-06 2.409664394691588e-06 -8.470329472543003e-22
1.7145235024755435e-06 1.7145235024755435e-06 0.0
3.0530375173598006e-07 3.053037517359801e-07 -5.293955920339377e-23
0.0 0.0 0.0
0.0 0.0 0.0
3.6030787873426255e-05 3.603078787342627e-05 -1.3552527156068805e-20
3.6650936559082564e-07 3.665093655908257e-07 -5.293955920339377e-23
3.1947807837682094e-07 3.194780783768209e-07 5.293955920339377e-23
1.235500384525891e-07 1.2355003845258913e-07 -2.6469779601696886e-23
1.5326466065634129e-06 1.5326466065634125e-06 4.235164736271502e-22
6.531773709682459e-07 6.531773709682458e-07 1.0587911840678754e-22
0.0 0.0 0.0
0.0 0.0 0.0
7.428975888384215e-08 7.428975888384215e-08 0.0
1.0608142319

## Scrap Code

In [130]:
for i in range(7):
    d['0'] = d['0'].dot(d['0'])

In [133]:
for index, row in d['0'].iterrows():
    for i in d['0'].columns:
        d['0'].iloc[index, i] = d['0'].iloc[index, i]*rowlis[index]

In [134]:
for index, row in d['0'].iterrows():
    print(sum(row))

0.00012811167991790386
2.6634506290340502e-06
2.983190018081867e-06
4.708407528389447e-07
8.033764508294475e-07
2.4096643946915785e-06
1.7145235024755478e-06
3.05303751735981e-07
0.0
0.0
3.603078787342636e-05
3.665093655908241e-07
3.194780783768217e-07
1.235500384525886e-07
1.532646606563407e-06
6.531773709682475e-07
0.0
0.0
7.428975888384233e-08
1.060814231976582e-06
2.6452169582927265e-06
8.923183983731826e-08
0.0
4.34462115919334e-07
1.531871497687559e-07
1.968701628190552e-07
0.0
3.2202828026159627e-07
1.8943396517858039e-07
4.346105286408698e-07
6.275367085835159e-09
1.855138455484333e-09
1.1115928805888127e-06
2.5435940693939322e-06
5.861739048612915e-07
1.0714558223657469e-07
5.5789103953230115e-05
1.2808661980723174e-07
5.092910691616315e-07
4.86943455663979e-09
3.9527298002101465e-07
5.435795993746166e-07
3.4637278442727745e-09
7.847172067733292e-05
1.0453349320098642e-06
9.610203712382718e-08
4.80563906122453e-07
9.101622074665082e-09
3.3042532632739896e-09
3.4541584419774497

In [119]:
for i in range(len(d['0'].columns)):
    d['0'].iloc[i,i]=0

In [120]:
d['0'].head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.000000,1.949968e-06,7.431052e-08,1.274318e-07,2.007954e-08,1.773606e-06,4.985412e-07,8.969325e-08,0.0,0.0,...,4.028123e-11,0.000012,5.829607e-07,2.097057e-09,0.0,0.0,1.552798e-08,5.700474e-07,0.000009,0.0
1,0.000018,0.000000e+00,2.064178e-08,5.994336e-08,4.325974e-07,1.482872e-05,2.253001e-07,4.241332e-08,0.0,0.0,...,1.010418e-11,0.000006,4.614075e-06,8.033442e-08,0.0,0.0,1.335945e-07,2.599051e-07,0.000005,0.0
2,0.000009,2.798123e-07,0.000000e+00,3.319264e-08,1.628984e-09,2.437555e-07,1.124505e-07,2.350143e-08,0.0,0.0,...,4.053407e-09,0.000003,8.362811e-08,1.266460e-10,0.0,0.0,2.083312e-09,1.428019e-07,0.000002,0.0
3,0.000039,1.931211e-06,8.083939e-08,0.000000e+00,1.956765e-08,1.755283e-06,4.989321e-07,8.976128e-08,0.0,0.0,...,3.607852e-11,0.000012,5.773041e-07,1.993049e-09,0.0,0.0,1.536223e-08,5.704935e-07,0.000009,0.0
4,0.000004,8.973855e-06,2.519007e-09,1.264912e-08,0.000000e+00,9.568495e-06,4.433434e-08,9.036006e-09,0.0,0.0,...,1.217175e-12,0.000001,3.024134e-06,5.248465e-06,0.0,0.0,8.298390e-08,5.199124e-08,0.000001,0.0


In [86]:
d['0'][0]

0     0.000000e+00
1     0.000000e+00
2     1.278286e-08
3     3.222322e-07
4     0.000000e+00
5     0.000000e+00
6     0.000000e+00
7     3.053038e-07
8     0.000000e+00
9     0.000000e+00
10    2.374677e-05
11    0.000000e+00
12    1.259282e-07
13    0.000000e+00
14    0.000000e+00
15    6.531774e-07
16    0.000000e+00
17    0.000000e+00
18    0.000000e+00
19    0.000000e+00
20    0.000000e+00
21    0.000000e+00
22    0.000000e+00
23    2.529847e-07
24    0.000000e+00
25    0.000000e+00
26    0.000000e+00
27    0.000000e+00
28    4.115991e-09
29    0.000000e+00
30    6.275367e-09
31    0.000000e+00
32    0.000000e+00
33    0.000000e+00
34    0.000000e+00
35    0.000000e+00
36    4.408503e-05
37    0.000000e+00
38    0.000000e+00
39    0.000000e+00
40    0.000000e+00
41    0.000000e+00
42    0.000000e+00
43    4.361603e-05
44    0.000000e+00
45    0.000000e+00
46    3.302315e-07
47    0.000000e+00
48    0.000000e+00
49    1.427026e-06
50    0.000000e+00
51    0.000000e+00
52    0.0000

In [53]:
d['0'] = d['0']/a

In [67]:
def convert_to_prob(row):
    a = sum(row)
    print(a)
    row = row/a

d['0'].apply(convert_to_prob, axis=1)


0.00012811167991790353
2.66345062903406e-06
2.9831900180818573e-06
4.708407528389434e-07
8.033764508294461e-07
2.409664394691588e-06
1.7145235024755435e-06
3.053037517359801e-07
0.0
0.0
3.603078787342627e-05
3.665093655908257e-07
3.194780783768209e-07
1.2355003845258913e-07
1.5326466065634125e-06
6.531773709682458e-07
0.0
0.0
7.428975888384215e-08
1.0608142319765895e-06
2.6452169582927215e-06
8.923183983731802e-08
0.0
4.344621159193329e-07
1.5318714976875532e-07
1.96870162819056e-07
0.0
3.220282802615959e-07
1.8943396517857986e-07
4.346105286408715e-07
6.275367085835144e-09
1.8551384554843295e-09
1.1115928805888102e-06
2.54359406939392e-06
5.861739048612934e-07
1.071455822365745e-07
5.578910395322998e-05
1.280866198072314e-07
5.092910691616309e-07
4.869434556639778e-09
3.952729800210138e-07
5.435795993746187e-07
3.4637278442727745e-09
7.847172067733271e-05
1.0453349320098722e-06
9.610203712382678e-08
4.805639061224518e-07
9.101622074665032e-09
3.3042532632739747e-09
3.454158441977442e-

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
40    None
41    None
42    None
43    None
44    None
45    None
46    None
47    None
48    None
49    None
50    None
51    None
52    None
53    None
54    None
55    None
56    None
57    None
dtype: object

In [52]:
for i in range(len(d['0'])):
    a = sum(d['0'][i])
    d['0'][i]/a

0.000388925937807693
